In [1]:
import random
import numpy as np
import scipy as sp
import collections
import pandas as pd
from pylab import *
import seaborn as sns
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output

sns.set_style('white')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1500)

In [2]:
def load_metabolomics(filename):
    # loading in TB plasma metabolomics data from tab-delimted file to pandas dataframe
    df = pd.read_csv(filename, sep='\t', lineterminator='\r') # loading data
    df['metabolite_name'] = df['metabolite_name'].str.strip('\n') # getting rid of line-terminator
    df = df.set_index('metabolite_name') # setting metabolite names as index
    df = df.transpose() 
    df.index.name = 'local_sample_id'
    df = df.reset_index()
    df.rename_axis(None, inplace=True)
    
    return df

In [3]:
def impute(df, thresh=0.1):
    null_allowed = len(df.index) * thresh
    null_columns = df.columns[df.isnull().sum() > null_allowed]
    
    print(null_columns)
    
    return df.dropna(axis=1)

In [4]:
def load_patientmetadata(filename, m_df):
    # mapping patient ID to group (control or case)

    # reading in patient metadata
    p_df = pd.read_excel(filename)
    p_df = p_df.set_index('local_sample_id')
    
    m_df = m_df.set_index('local_sample_id').join(p_df)
    m_df.columns = m_df.columns.str.lower()
    #sampleIDtoClass = p_df.set_index('local_sample_id').to_dict()['GROUP'] # creating dictionary

    # Mapping data
    #m_df['Group'] = m_df.index.map(sampleIDtoClass)
    #groupAssignments_list = list(m_df['Group'])
    
    return m_df, p_df

In [5]:
# standardizing data by making values (features) zero-mean and unit-variance
def standardize_data(f_vals):
    from sklearn import preprocessing

    # applying standardization 
    standardizerScaler = preprocessing.StandardScaler()
    data_StandardScaled = standardizerScaler.fit_transform(f_vals)
    
    return data_StandardScaled

In [6]:
def make_df(f_vals, features, l_vals, labels):
    df = pd.concat([pd.DataFrame(data=l_vals, columns=labels), 
                    pd.DataFrame(data=f_vals, columns=features)], axis=1)
    return df

In [7]:
def perform_PCA(data, l_vals, labels, save=False, ncomp=10):
# computing principal components
    from sklearn import decomposition

    pcaAbs = decomposition.PCA(n_components=ncomp)
    data_PCA = pcaAbs.fit_transform(data)
    
    pc_cols = ['PC ' + str(i) for i in np.arange(1, ncomp + 1)]
    df_PCA = make_df(data_PCA, pc_cols, l_vals, labels)
    
    #Plot explained variance by number of components
    var_exp = pcaAbs.explained_variance_ratio_
    fig_ve, ax_ve = plt.subplots(1, 1)
    sns.lineplot(x=(np.arange(len(var_exp)) + 1), y=np.cumsum(var_exp), ax=ax_ve)
    plt.xlabel('PCA component number')
    plt.ylabel('Cumulative variance ratio')
    if save:
        plt.savefig('variance-exp.png', bbox_inches='tight', pad_inches=0.5)
    
    fig_pca, ax_pca = plt.subplots(1, 1)
    sns.scatterplot(x='PC 1', y='PC 2', data=df_PCA, hue='group', ax=ax_pca)
    
    return df_PCA

In [ ]:
os.chdir('.\data')
metabolomicsTB_df = load_metabolomics('Metabolomics_Data.txt')
metabolomicsTB_df, patient_df = load_patientmetadata('Patient_Metadata.xlsx', metabolomicsTB_df)
metabolomicsTB_df = impute(metabolomicsTB_df)

# displaying shape and first few data entries
print('The shape of our data matrix is: ', metabolomicsTB_df.shape)
metabolomicsTB_df.head()


In [ ]:
labels = ['group', 'mb_sample_id', 'timepoint', 'region', 'gender']
features = [x for x in metabolomicsTB_df.columns.to_list() if x not in labels]
f_vals = metabolomicsTB_df.loc[:, features].values
l_vals = metabolomicsTB_df.loc[:, labels].values


std_data = standardize_data(f_vals)

std_df = make_df(std_data, features, l_vals, labels)
# displaying shape and first few data entries
print('The shape of the standardized data is:', std_df.shape)
display(std_df.head())

In [ ]:
males_data = std_data[ std_df['timepoint'].str.contains('M')]
males_labels = l_vals[ std_df['timepoint'].str.contains('M')]
pca_df = perform_PCA(males_data, males_labels, labels, save=False, ncomp=10)
display(pca_df.head())

In [ ]:
metabolites = list(metabolomicsTB_df.columns)[:-1]
numMetabas = metabolomicsTB_df.shape[1] - 1
alpha = 0.05

significant = []
for metab in metabolites:
    indicesControl = metabolomicsTB_df['group'] == 'control'
    indicesCase = metabolomicsTB_df['group'] == 'case'
    
    metabAverageControl = list(metabolomicsTB_df.loc[indicesControl, metab])
    metabAverageCase = list(metabolomicsTB_df.loc[indicesCase, metab])
    stat, p_val = sp.stats.ranksums(metabAverageControl, metabAverageCase)
    
    if p_val  < alpha / numMetabas:
        significant.append(metab)

In [ ]:
significant

In [ ]:
#Do Spearman correlation for healthy vs. disease
display(metabolomicsTB_df.sort_values(by=['timepoint', 'group']))
#df.corr(method='spearman')

In [10]:
#os.chdir('.\data')
ut_file = os.path.join('.\data\met_data', 'ST000974_AN001595_res.txt')
ut_met = pd.read_csv(ut_file, sep='\t', lineterminator='\r')
ut_met['Samples'] = ut_met['Samples'].str.strip('\n')
metadata = ut_met['group'].str.rsplit(' | ', expand=True)
cols = metadata.iloc[0, :].str.rsplit(':', expand=True)[0].str.lower()
metadata.columns = cols
metadata = metadata.apply(lambda x: x.str.rsplit(':', expand=True)[1])

ut_met = ut_met.drop(columns='group')
ut_met = ut_met.join(metadata)
ut_met.dropna(axis=0)
#display(metadata)
#display(ut_met)



,Samples,X - 07765,X - 09789,X - 10346,X - 10445,X - 10458,X - 11261,X - 11299,X - 11308,X - 11315,X - 11378,X - 11381,X - 11429,X - 11437,X - 11438,X - 11440,X - 11441,X - 11442,X - 11444,X - 11470,X - 11478,X - 11483,X - 11491,X - 11522,X - 11530,X - 11537,X - 11538,X - 11632,X - 11687,X - 11787,X - 11795,X - 11837,X - 11838,X - 11843,X - 11847,X - 11849,X - 11850,X - 11852,X - 11858,X - 11871,X - 11880,X - 11905,X - 11945,X - 12007,X - 12013,X - 12026,X - 12039,X - 12056,X - 12063,X - 12093,X - 12095,X - 12096,X - 12097,X - 12100,X - 12101,X - 12104,X - 12108,X - 12114,X - 12115,X - 12125,X - 12127,X - 12170,X - 12199,X - 12206,X - 12212,X - 12216,X - 12230,X - 12257,X - 12261,X - 12283,X - 12329,X - 12386,X - 12407,X - 12411,X - 12450,X - 12462,X - 12472,X - 12543,X - 12544,X - 12686,X - 12689,X - 12690,X - 12714,X - 12718,X - 12726,X - 12729,X - 12730,X - 12734,X - 12739,X - 12740,X - 12748,X - 12798,X - 12812,X - 12816,X - 12818,X - 12821,X - 12822,X - 12833,X - 12844,X - 12847,X - 12849,X - 12850,X - 13007,X - 13152,X - 13435,X - 13478,X - 13507,X - 13529,X - 13553,X - 13658,X - 13727,X - 13728,X - 13729,X - 13730,X - 13737,X - 13835,X - 13837,X - 13848,X - 13862,X - 13866,X - 13891,X - 14105,X - 14141,X - 14196,X - 14213,X - 14291,X - 14302,X - 14314,X - 14318,X - 14320,X - 14347,X - 14352,X - 14374,X - 14384,X - 14465,X - 14473,X - 14568,X - 14625,X - 14626,X - 14658,X - 14662,X - 14665,X - 14697,X - 14904,X - 14961,X - 15136,X - 15150,X - 15192,X - 15220,X - 15245,X - 15375,X - 15382,X - 15461,X - 15469,X - 15477,X - 15484,X - 15486,X - 15492,X - 15497,X - 15502,X - 15503,X - 15562,X - 15563,X - 15636,X - 15646,X - 15664,X - 15678,X - 15728,X - 15807,X - 15824,X - 15863,X - 16124,X - 16129,X - 16134,X - 16245,X - 16271,X - 16343,X - 16397,X - 16480,X - 16564,X - 16570,X - 16576,X - 16581,X - 16653,X - 16654,X - 16935,X - 16938,X - 16940,X - 16944,X - 16946,X - 16947,X - 16964,X - 16975,X - 17137,X - 17138,X - 17145,X - 17185,X - 17269,X - 17299,X - 17305,X - 17306,X - 17320,X - 17327,X - 17339,X - 17340,X - 17346,X - 17348,X - 17354,X - 17365,X - 17367,X - 17438,X - 17469,X - 17653,X - 17654,X - 17674,X - 17676,X - 17690,X - 17702,X - 17717,X - 17801,X - 17866,X - 17969,X - 18039,X - 18165,X - 18241,X - 18249,X - 18345,X - 18446,X - 18779,X - 18887,X - 18889,X - 18901,X - 18913,X - 18914,X - 18921,X - 18922,X - 19141,X - 19183,X - 19197,X - 19299,X - 19350,X - 19437,X - 19438,X - 19455,X - 19561,X - 19823,X - 20529,X - 20587,X - 20598,X - 20731,X - 20755,X - 21258,X - 21285,X - 21286,X - 21295,X - 21310,X - 21312,X - 21315,X - 21319,X - 21328,X - 21333,X - 21339,X - 21341,X - 21342,X - 21343,X - 21346,X - 21353,X - 21358,X - 21364,X - 21365,X - 21408,X - 21431,X - 21437,X - 21441,X - 21442,X - 21448,X - 21458,X - 21461,X - 21465,X - 21470,X - 21471,X - 21473,X - 21474,X - 21583,X - 21607,X - 21658,X - 21661,X - 21666,X - 21668,X - 21729,X - 21733,X - 21736,X - 21737,X - 21740,X - 21757,X - 21769,X - 21774,X - 21788,X - 21792,X - 21796,X - 21801,X - 21802,X - 21803,X - 21807,X - 21810,X - 21816,X - 21821,X - 21830,X - 21834,X - 21839,X - 21842,X - 21844,X - 21845,X - 21849,X - 21851,X - 21891,X - 21892,X - 21935,X - 21959,X - 22083,X - 22102,X - 22154,X - 22172,X - 22173,X - 22174,X - 22414,X - 22415,X - 22507,X - 22508,X - 22509,X - 22515,X - 22518,X - 22519,X - 22520,X - 22585,X - 22586,X - 22590,group,timepoint,region,gender
86,MPIIB.5369,0.07,0.69,0.80,0.05,1.16,1.41,0.07,1.22,0.52,1.49,0.24,0.80,6.13,0.54,2.25,0.70,0.87,1.39,1.44,1.53,0.07,0.43,0.71,0.99,0.07,0.77,1.30,0.49,0.37,1.02,0.58,0.01,0.74,0.54,0.14,1.12,0.06,0.00,0.17,0.76,0.66,0.25,1.55,0.61,0.25,0.01,0.45,0.70,0.05,0.93,0.06,0.04,0.10,0.03,0.02,0.07,0.18,0.04,0.08,0.15,0.85,0.97,1.26,1.31,0.09,1.38,0.18,0.34,0.11,0.07,1.11,0.32,0.04,1.39,1.29,0.24,0.11,49.40,0.72,0.20,0.15,2.14,0.16,2.88,0.11,0.25,0.18,0.40,0.13,0.44,0.14,0.04,0.01,0.09,0.01,0.62,0.75,0.86,0.02,0.01,3.18,1.49,0.12,0.82,0.18,0.08,0.12,0.40,1.27,0.06,0.01,0.04,0.09,0.10,0.24,0.88,0.01,0

In [11]:
ut_met.columns = (ut_met.columns.str.lstrip('X - ')
                                .str.lstrip('0')
                                .str.lower())
ut_met = ut_met.fillna(ut_met.min())  #impute missing values as = limit of detection
ut_met.head()



,samples,7765,9789,10346,10445,10458,11261,11299,11308,11315,11378,11381,11429,11437,11438,11440,11441,11442,11444,11470,11478,11483,11491,11522,11530,11537,11538,11632,11687,11787,11795,11837,11838,11843,11847,11849,11850,11852,11858,11871,11880,11905,11945,12007,12013,12026,12039,12056,12063,12093,12095,12096,12097,12100,12101,12104,12108,12114,12115,12125,12127,12170,12199,12206,12212,12216,12230,12257,12261,12283,12329,12386,12407,12411,12450,12462,12472,12543,12544,12686,12689,12690,12714,12718,12726,12729,12730,12734,12739,12740,12748,12798,12812,12816,12818,12821,12822,12833,12844,12847,12849,12850,13007,13152,13435,13478,13507,13529,13553,13658,13727,13728,13729,13730,13737,13835,13837,13848,13862,13866,13891,14105,14141,14196,14213,14291,14302,14314,14318,14320,14347,14352,14374,14384,14465,14473,14568,14625,14626,14658,14662,14665,14697,14904,14961,15136,15150,15192,15220,15245,15375,15382,15461,15469,15477,15484,15486,15492,15497,15502,15503,15562,15563,15636,15646,15664,15678,15728,15807,15824,15863,16124,16129,16134,16245,16271,16343,16397,16480,16564,16570,16576,16581,16653,16654,16935,16938,16940,16944,16946,16947,16964,16975,17137,17138,17145,17185,17269,17299,17305,17306,17320,17327,17339,17340,17346,17348,17354,17365,17367,17438,17469,17653,17654,17674,17676,17690,17702,17717,17801,17866,17969,18039,18165,18241,18249,18345,18446,18779,18887,18889,18901,18913,18914,18921,18922,19141,19183,19197,19299,19350,19437,19438,19455,19561,19823,20529,20587,20598,20731,20755,21258,21285,21286,21295,21310,21312,21315,21319,21328,21333,21339,21341,21342,21343,21346,21353,21358,21364,21365,21408,21431,21437,21441,21442,21448,21458,21461,21465,21470,21471,21473,21474,21583,21607,21658,21661,21666,21668,21729,21733,21736,21737,21740,21757,21769,21774,21788,21792,21796,21801,21802,21803,21807,21810,21816,21821,21830,21834,21839,21842,21844,21845,21849,21851,21891,21892,21935,21959,22083,22102,22154,22172,22173,22174,22414,22415,22507,22508,22509,22515,22518,22519,22520,22585,22586,22590,group,timepoint,region,gender
0,MPIIB.9274,0.05,0.36,0.86,1.71,0.81,1.06,0.26,1.26,0.75,1.61,1.78,1.02,3.91,10.72,0.77,0.33,0.36,0.88,0.79,3.25,1.25,0.15,0.60,0.59,1.11,0.40,1.18,1.01,0.96,0.28,3.03,0.01,2.48,0.01,0.04,3.70,0.06,0.00,1.02,0.66,0.56,0.85,0.08,5.14,1.13,1.02,0.05,0.24,0.98,0.50,0.06,0.04,0.85,1.96,1.06,1.01,0.18,0.61,0.94,0.48,0.05,0.18,0.85,0.21,0.99,1.54,3.74,0.98,0.52,0.07,0.52,0.07,0.17,3.54,1.27,5.90,0.29,0.11,0.98,1.26,0.93,0.09,2.35,0.36,0.65,0.11,0.14,1.86,0.13,1.92,0.14,0.04,3.86,0.09,0.01,0.34,0.03,1.20,0.19,0.01,0.37,0.62,0.93,4.34,2.17,1.69,0.57,0.72,0.98,0.34,2.26,0.56,0.09,0.88,0.05,1.88,0.01,0.16,0.74,2.07,0.28,1.75,1.43,0.88,0.96,0.02,1.11,0.72,1.81,0.33,0.77,0.90,1.34,1.50,0.98,0.91,0.87,1.35,1.07,0.12,0.02,0.27,0.10,0.84,1.18,1.11,0.62,1.04,0.26,1.33,1.36,1.31,1.31,1.20,1.41,1.20,0.47,0.32,0.22,0.53,1.29,1.31,0.03,0.09,0.09,1.25,0.40,1.13,0.01,0.74,0.07,1.24,0.47,1.22,0.04,1.74,2.23,0.97,2.77,1.86,0.86,0.84,1.06,0.02,1.95,0.94,0.20,3.45,0.70,0.06,1.32,0.32,0.13,1.59,1.02,0.56,1.46,0.90,0.14,0.25,0.10,5.56,1.00,0.68,0.38,0.21,0.98,0.42,2.05,0.90,2.42,0.61,0.27,0.17,1.15,0.70,0.05,0.61,0.06,0.04,0.94,0.05,1.54,0.08,2.23,0.64,0.94,10.51,0.40,0.68,1.78,1.51,1.60,4.29,0.05,0.70,0.05,0.04,0.06,1.07,1.03,0.46,0.49,0.14,1.38,0.45,0.74,0.56,0.19,0.50,0.86,1.94,0.99,0.05,1.56,0.53,0.06,1.31,0.82,1.30,1.44,0.02,1.20,6.77,3.23,5.23,2.44,1.35,2.07,1.12,4.50,1.09,1.40,6.76,0.70,1.05,1.21,0.10,1.42,0.73,0.63,1.98,0.94,0.87,1.71,0.04,0.61,0.62,1.01,0.13,1.75,0.02,0.16,1.05,0.52,4.25,0.72,1.03,0.92,3.17,0.45,0.02,0.20,0.86,0.12,0.49,0.86,0.05,3.41,0.44,0.15,0.72,0.58,1.22,0.95,1.90,0.16,9.25,0.03,0.42,0.75,0.44,1.02,0.13,1.39,0.95,1.20,1.03,1.07,0.06,0.87,1.24,0.33,0.13,0.05,0.11,case,BL,AHRI,F
1,MPIIB.8120,0.70,0.53,0.01,0.59,0.67,2.12,0.14,1.07,0.90,1.06,1.39,1.09,1.78,1.82,0.82,0.13,0.10,1.03,0.54,1.09,0.51,0.38,0.26,0.29,0.48,1.00,0.53,0.73,0.97,0.70,0.14,0.01,0.00,0.00,0.01,0.02,0.06,0.00,0.30,0.46,1.25,0.67,7.89,0.11,1.06

In [12]:
labels = ['samples', 'timepoint', 'region', 'gender']
mz = [x for x in ut_met.columns.to_list() if x not in labels]
ut_mz = ut_met[mz]
ut_met_summary = ut_mz.groupby(['group']).agg([('Mean', np.nanmean), ('Std', np.nanstd)])
display(ut_met_summary)

7765      9789      10346      10445      10458      11261      11299  \
        Mean  Std Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                           
case    2.18 3.21 1.23 1.14  1.11 1.88  0.46 1.18  1.22 0.95  1.32 1.32  0.25   
control 1.88 3.09 1.22 0.83  1.45 4.06  0.31 0.70  1.56 2.07  1.30 1.49  0.32   

             11308      11315      11378      11381      11429      11437  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    0.47  1.00 0.45  1.08 0.54  0.97 0.59  1.28 1.27  1.14 0.45  5.37   
control 0.76  1.08 0.48  1.28 1.45  1.12 0.59  1.32 1.18  1.01 0.23  5.23   

              11438      11440      11441      11442      11444      11470  \
          Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                        
case    11.15  1.30 1.33  1.12 0.79  1.26 1.29  1.36 1.45  0.98 0.55  1.17   
control 15.56  1.22 1.00  1.12 0.76  1.44 1.42  1.60 2.02  1.12 0.60  1.16   

             11478      11483      11491      11522      11530      11537  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    0.81  1.45 1.43  0.28 0.51  1.60 3.03  1.23 1.23  1.24 1.20  0.52   
control 0.86  1.72 3.50  0.36 0.79  1.57 1.94  1.45 1.46  1.45 1.40  0.61   

             11538      11632      11687      11787      11795      11837  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    0.73  1.04 0.51  1.23 1.30  1.15 0.73  1.02 0.34  1.19 1.10  1.63   
control 1.19  1.17 0.68  1.54 2.29  1.08 0.84  1.03 0.29  1.19 0.89  1.48   

             11838       11843      11847      11849      11850      11852  \
         Std  Mean   Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                        
case    1.99  6.15 56.99  1.07 1.25  0.18 0.76  0.24 1.02  1.13 1.18  1.47   
control 1.71  0.74  4.28  0.92 0.95  0.43 1.78  0.99 5.62  1.15 1.27  0.95   

             11858      11871      11880      11905      11945      12007  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    5.69  0.14 0.68  0.56 0.82  0.97 0.55  1.10 0.57  0.53 0.43  1.27   
control 2.55  0.30 1.48  0.65 1.48  1.10 0.56  1.13 0.62  0.51 0.43  1.57   

             12013      12026      12039      12056       12063      12093  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean   Std  Mean  Std  Mean   
group                                                                        
case    1.74  1.42 1.55  0.54 0.52  1.28 2.01  9.09 32.88  1.02 0.81  0.85   
control 2.79  1.26 1.45  0.49 0.42  1.75 2.92  7.71 34.31  1.33 1.16  0.99   

             12095      12096      12097      12100      12101      12104  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    1.90  1.43 1.29  0.44 1.09  0.39 1.04  1.14 0.58  0.51 0.91  1.15   
control 1.63  1.11 0.78  1.03 3.31  0.87 3.36  1.13 0.72  0.43 0.84  0.99   

             12108      12114      12115      12125      12127      12170  \
         Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean  Std  Mean   
group                                                                       
case    0.67  0.45 0.96  0.33 0.39  0.35 0.89  0.52 0.86  1.45 3.59  1.20   
control 0.45  0.99 2.66  0.40 0.67  0.23 0.56  0.52 1.09  0.65 1.09  1.09   

             12199      12206      12212      12216      12230      12257  \
         Std  Mean  Std  Mean  Std  

In [20]:
metabolites = mz[:-1]
#print(metabolites)
numMetabas = len(mz)
alpha_diff = 0.05
alpha_normal = 0.05
significant = []
for metab in metabolites:
    indicesControl = np.array(ut_met['group'] == 'control') * np.array(ut_met['timepoint'] != 'BL')
    indicesCase = np.array(ut_met['group'] == 'case') * np.array(ut_met['timepoint'] != 'BL')
    
    
    metabAverageControl = list(ut_met.loc[indicesControl, metab])
    metabAverageCase = list(ut_met.loc[indicesCase, metab])
    
    
    _, p_normal_ctrl = sp.stats.normaltest(metabAverageControl, nan_policy='omit')
    _, p_normal_case = sp.stats.normaltest(metabAverageCase, nan_policy='omit')
    #print(metab, p_normal_ctrl, p_normal_case)
    
    if (p_normal_ctrl < alpha_normal and p_normal_case < alpha_normal):
        _, p_var = sp.stats.bartlett(metabAverageControl, metabAverageCase)
        _, p_diff = sp.stats.ttest_ind(metabAverageControl, metabAverageCase, nan_policy='omit', equal_var=(p_var<alpha_normal))
    else:
        _, p_diff = sp.stats.ranksums(metabAverageControl, metabAverageCase)
    
    if p_diff  < alpha_diff:
        significant.append(float(metab))
        
#print(significant)
significant = pd.DataFrame({'mz' : significant})
display(significant)

significant.to_csv('significant_mz.csv')

,mz
0,"11,444.00"
1,"11,687.00"
2,"12,063.00"
3,"12,095.00"
4,"12,104.00"
5,"12,127.00"
6,"12,689.00"
7,"12,690.00"
8,"12,748.00"
9,"12,850.00"


In [14]:
##ADAPTED FROM CDUVALLET
#positive ion mode
def calculate_adducts(mzs):
    m = {'C': 12.0,
         'H': 1.007825,
         'O': 15.994915,
         'N': 14.003074,
         'S': 31.972072,
         'Cl': 34.968853,
         'F': 18.998403,
         'B': 10.012938,
         'D': 2.014102,
         'Na': 22.989770,
         'K' : 39.0983, 
         'e-': 0.000548579909}
    
    plush = [(float(i) - m['H']) for i in mzs]
    plusnh4 = [(float(i) - (m['N'] + 4 * m['H'])) for i in mzs] 
    plush2o = [(float(i) - (2 * m['H'] + m['O'])) for i in mzs]
    plusna = [(float(i) - m['Na']) for i in mzs]
    plusk = [(float(i) - m['K']) for i in mzs]
    plusch3cn = [(float(i) - (2 * m['C'] + 3 * m['H'] + m['N'])) for i in mzs]
    return plush, plusnh, plush2o, plusna, plusk, plusch3cn

def parse_HMDB(xml_file):
    from lxml import etree
    ## ADAPTED FROM CDUVALLET
    outdict = {}
    tree = etree.iterparse(xml_file, tag='metabolite')
    counter = 0

    for event, elem in tree:
        counter += 1
        if counter % 2000 == 0:
            print('Reading {}th metabolite from HMDB'.format(counter))
        tmpdict = {}
        tmpdict['name'] = elem.findtext('name')
        if elem.findtext('monisotopic_moleculate_weight'):
            tmpdict['neutral_mass'] = elem.findtext('monisotopic_moleculate_weight')
        else:
            tmpdict['neutral_mass'] = '0.0'

        outdict[elem.findtext('accession')] = tmpdict
        elem.clear()
    return outdict



In [ ]:
xml_file = os.path.join('.\data\met_data', 'cleaned_serum_metabolites.xml')
outdict = parse_HMDB(xml_file)
[]